In [43]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import sqlite3, pandas as pd
from contextlib import closing

DB_PATH = "/Users/farazahmed/Documents/GitHub/steam-discount-forecast/data/steam_sales.db"

In [41]:
with closing(sqlite3.connect(DB_PATH)) as conn:
    df = pd.read_sql("""
        WITH scoped_games AS (
            SELECT g.itad_id, g.appid, g.title, g.release_date,
                   g.early_access, g.mature
            FROM games g
            WHERE g.type='game'
              AND g.release_date BETWEEN '2021-01-01' AND '2024-12-31'
        ),
        tags_agg AS (
            SELECT gt.itad_id, GROUP_CONCAT(t.name, '; ') AS tags
            FROM game_tags gt JOIN tags t ON t.tag_id=gt.tag_id
            GROUP BY gt.itad_id
        ),
        first_hist_after_release AS (
            SELECT h.itad_id, MIN(h.ts_utc) AS first_ts
            FROM history_events h
            JOIN scoped_games sg ON sg.itad_id=h.itad_id
            WHERE date(h.ts_utc) >= date(sg.release_date)
            GROUP BY h.itad_id
        ),
        launch_price AS (
            SELECT fhar.itad_id, he.regular AS launch_price
            FROM first_hist_after_release fhar
            JOIN history_events he
              ON he.itad_id=fhar.itad_id AND he.ts_utc=fhar.first_ts
        ),
        first_sale_pick AS (
            SELECT h.itad_id, MIN(h.ts_utc) AS first_sale_ts
            FROM history_events h
            JOIN scoped_games sg ON sg.itad_id=h.itad_id
            WHERE date(h.ts_utc) > date(sg.release_date)
              AND h.price < h.regular    -- any discount, no 5% cutoff
            GROUP BY h.itad_id
        ),
        first_sale_enriched AS (
            SELECT fsp.itad_id,
                   fsp.first_sale_ts,
                   date(fsp.first_sale_ts) AS first_sale_date,
                   he.cut    AS first_sale_cut,
                   he.price  AS first_sale_price,
                   he.regular AS first_sale_regular
            FROM first_sale_pick fsp
            JOIN history_events he
              ON he.itad_id=fsp.itad_id AND he.ts_utc=fsp.first_sale_ts
        ),
        first_sale_labeled AS (
            SELECT fse.*,
                   CAST(ROUND(julianday(date(fse.first_sale_ts)) - julianday(g.release_date)) AS INTEGER)
                     AS days_to_first_sale
            FROM first_sale_enriched fse
            JOIN scoped_games g ON g.itad_id=fse.itad_id
        )
        SELECT
            sg.itad_id, sg.appid, sg.title, sg.release_date,
            sg.early_access, sg.mature,
            lp.launch_price,
            fsl.first_sale_date, fsl.first_sale_cut,
            fsl.first_sale_price, fsl.first_sale_regular,
            fsl.days_to_first_sale,
            ta.tags
        FROM scoped_games sg
        LEFT JOIN launch_price lp     ON lp.itad_id = sg.itad_id
        LEFT JOIN first_sale_labeled fsl ON fsl.itad_id = sg.itad_id
        LEFT JOIN tags_agg ta        ON ta.itad_id = sg.itad_id
        WHERE fsl.days_to_first_sale IS NOT NULL;
    """, conn)

df.head()

,itad_id,appid,title,release_date,early_access,mature,launch_price,first_sale_date,first_sale_cut,first_sale_price,first_sale_regular,days_to_first_sale,tags
0,018d937e-e9b7-73a9-84e1-1546a85c9841,NaN,Hacker,2024-03-29,0,0,0.99,2024-05-13,51,0.49,0.99,45,None
1,018d937e-e9ba-71b7-b901-de864dd7397c,1040510.0,Princess of Zeven,2023-10-28,0,1,12.99,2023-12-21,20,10.39,12.99,54,RPG; Indie; Nudity; Sexual Content; Female Pro...
2,018d937e-e9c0-7185-a3a5-de3b8bff7956,832360.0,Hotel Magnate,2021-10-05,1,0,24.99,2021-11-24,20,19.99,24.99,50,Simulation; Sandbox; Management; Building; Res...
3,018d937e-e9c1-71fc-9f4a-472d6505c3cb,2361080.0,Step by Step,2023-10-06,0,0,9.99,2023-11-10,20,7.99,9.99,35,Hand-drawn; Casual; Platformer; 2D Platformer;...
4,018d937e-e9cb-728b-8309-979905bf3e82,965990.0,Destiny's Sword,2022-10-17,1,0,8.99,2022-12-22,40,5.39,8.99,66,Adventure; Simulation; Choices Matter; Sci-fi;...


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31246 entries, 0 to 31245
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itad_id             31246 non-null  object 
 1   appid               31118 non-null  float64
 2   title               31246 non-null  object 
 3   release_date        31246 non-null  object 
 4   early_access        31246 non-null  int64  
 5   mature              31246 non-null  int64  
 6   launch_price        31246 non-null  float64
 7   first_sale_date     31246 non-null  object 
 8   first_sale_cut      31246 non-null  int64  
 9   first_sale_price    31246 non-null  float64
 10  first_sale_regular  31246 non-null  float64
 11  days_to_first_sale  31246 non-null  int64  
 12  tags                31118 non-null  object 
dtypes: float64(4), int64(4), object(5)
memory usage: 3.1+ MB


In [50]:
df.isna().sum()

itad_id                 0
appid                 128
title                   0
release_date            0
early_access            0
mature                  0
launch_price            0
first_sale_date         0
first_sale_cut          0
first_sale_price        0
first_sale_regular      0
days_to_first_sale      0
tags                  128
dtype: int64

In [51]:
# dropping the rowas with missing appid

df = df[~df['appid'].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31118 entries, 1 to 31245
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itad_id             31118 non-null  object 
 1   appid               31118 non-null  float64
 2   title               31118 non-null  object 
 3   release_date        31118 non-null  object 
 4   early_access        31118 non-null  int64  
 5   mature              31118 non-null  int64  
 6   launch_price        31118 non-null  float64
 7   first_sale_date     31118 non-null  object 
 8   first_sale_cut      31118 non-null  int64  
 9   first_sale_price    31118 non-null  float64
 10  first_sale_regular  31118 non-null  float64
 11  days_to_first_sale  31118 non-null  int64  
 12  tags                31118 non-null  object 
dtypes: float64(4), int64(4), object(5)
memory usage: 3.3+ MB


#### The dataset from Kaggle contains some other useful features like "Age", "Achievements", "Categories", etc. I will join these features into my data.

In [45]:
df2 = pd.read_csv("/Users/farazahmed/Documents/GitHub/steam-discount-forecast/Kaggle datasets/93182_steam_games.csv")
df2.head()

/var/folders/1f/b537t8w54jj9x40zq_pnqq4c0000gn/T/ipykernel_89105/2777268084.py:1: DtypeWarning: Columns (3,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/Users/farazahmed/Documents/GitHub/steam-discount-forecast/Kaggle datasets/93182_steam_games.csv")


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1424640,余烬,"Oct 3, 2020",20000 - 50000,0,0,3.99,0,'Ashes of war' is an anti war theme adventure ...,['Simplified Chinese'],...,0,0,0,宁夏华夏西部影视城有限公司,宁夏华夏西部影视城有限公司,"Single-player,Family Sharing","Adventure,Casual,Indie,RPG","Sokoban,RPG,Puzzle-Platformer,Exploration,Adve...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
1,402890,Nyctophilia,"Sep 23, 2015",50000 - 100000,0,0,0.00,0,NYCTOPHILIA Nyctophilia is an 2D psychological...,"['English', 'Russian']",...,0,0,0,Cat In A Jar Games,Cat In A Jar Games,Single-player,"Adventure,Free To Play,Indie","Free to Play,Indie,Adventure,Horror,2D,Pixel G...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
2,1151740,Prison Princess,"Apr 2, 2020",0 - 20000,0,0,19.99,0,"ABOUT Now nothing more than a phantom, can the...","['English', 'Simplified Chinese', 'Traditional...",...,0,0,0,qureate,qureate,"Single-player,Steam Achievements,Full controll...","Adventure,Indie","Sexual Content,Adventure,Indie,Nudity,Anime,Ma...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
3,875530,Dead In Time,"Oct 12, 2018",0 - 20000,0,0,7.99,0,Is a hardcore action with a non-trivial level ...,"['English', 'Russian']",...,0,0,0,Zelenov Artem,Zelenov Artem,"Single-player,Full controller support,Family S...","Action,Indie","Action,Indie,Souls-like,Fantasy,Early Access,R...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
4,1835360,Panacle: Back To Wild,"Mar 11, 2022",0 - 20000,2,0,3.99,0,Panacle: Back to the Wild is a indie card game...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0,0,0,渡鸦游戏,"渡鸦游戏,电钮组","Single-player,Family Sharing","Indie,Strategy,Early Access","Trading Card Game,Turn-Based Strategy,Lore-Ric...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [56]:
# renaming "AppID" column to "appid" to match with df

df2.rename(columns={"AppID": "appid"}, inplace=True)

In [57]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93182 entries, 0 to 93181
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   appid                       93182 non-null  int64  
 1   Name                        93179 non-null  object 
 2   Release date                93182 non-null  object 
 3   Estimated owners            16462 non-null  object 
 4   Peak CCU                    93182 non-null  int64  
 5   Required age                93182 non-null  int64  
 6   Price                       93182 non-null  float64
 7   DLC count                   93182 non-null  int64  
 8   About the game              88392 non-null  object 
 9   Supported languages         93182 non-null  object 
 10  Full audio languages        93182 non-null  object 
 11  Reviews                     10599 non-null  object 
 12  Header image                93182 non-null  object 
 13  Website                     416

In [62]:
# merging the two dataframes on "appid" and keeping the columns "Required Age", "Windoes", "Mac", "Linux", "Achievements", "Developer", "Publisher", "Categories", "Genres"

df_merged = pd.merge(df, df2[['appid', 'Required age', 'Windows', 'Mac', 'Linux', 'Achievements', 'Developers', 'Publishers', 'Categories', 'Genres']], on='appid', how='left')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31118 entries, 0 to 31117
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itad_id             31118 non-null  object 
 1   appid               31118 non-null  float64
 2   title               31118 non-null  object 
 3   release_date        31118 non-null  object 
 4   early_access        31118 non-null  int64  
 5   mature              31118 non-null  int64  
 6   launch_price        31118 non-null  float64
 7   first_sale_date     31118 non-null  object 
 8   first_sale_cut      31118 non-null  int64  
 9   first_sale_price    31118 non-null  float64
 10  first_sale_regular  31118 non-null  float64
 11  days_to_first_sale  31118 non-null  int64  
 12  tags                31118 non-null  object 
 13  Required age        31115 non-null  float64
 14  Windows             31115 non-null  object 
 15  Mac                 31115 non-null  object 
 16  Linu

In [63]:
df_merged.isna().sum()

itad_id                 0
appid                   0
title                   0
release_date            0
early_access            0
mature                  0
launch_price            0
first_sale_date         0
first_sale_cut          0
first_sale_price        0
first_sale_regular      0
days_to_first_sale      0
tags                    0
Required age            3
Windows                 3
Mac                     3
Linux                   3
Achievements            3
Developers             32
Publishers            136
Categories              5
Genres                 34
dtype: int64

In [64]:
# dropping the rows with missing "Required age", "Windows", "Mac", "Linux", "Achievements", "Developer", "Publisher", "Categories", "Genres"
df_merged = df_merged[~df_merged['Required age'].isna()]
df_merged = df_merged[~df_merged['Windows'].isna()]
df_merged = df_merged[~df_merged['Mac'].isna()]
df_merged = df_merged[~df_merged['Linux'].isna()]
df_merged = df_merged[~df_merged['Achievements'].isna()]
df_merged = df_merged[~df_merged['Developers'].isna()]
df_merged = df_merged[~df_merged['Publishers'].isna()]
df_merged = df_merged[~df_merged['Categories'].isna()]
df_merged = df_merged[~df_merged['Genres'].isna()]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30947 entries, 0 to 31117
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itad_id             30947 non-null  object 
 1   appid               30947 non-null  float64
 2   title               30947 non-null  object 
 3   release_date        30947 non-null  object 
 4   early_access        30947 non-null  int64  
 5   mature              30947 non-null  int64  
 6   launch_price        30947 non-null  float64
 7   first_sale_date     30947 non-null  object 
 8   first_sale_cut      30947 non-null  int64  
 9   first_sale_price    30947 non-null  float64
 10  first_sale_regular  30947 non-null  float64
 11  days_to_first_sale  30947 non-null  int64  
 12  tags                30947 non-null  object 
 13  Required age        30947 non-null  float64
 14  Windows             30947 non-null  object 
 15  Mac                 30947 non-null  object 
 16  Linux    

#### From the users perspective it is not possible for them to have the data like: "first_sale_date", "first_sale_cut", "first_sale_price", "first_sale_regular". So it does not make sense to have them in the model.

In [ ]:
# dropping the columns that are not needed for the analysis

df_final = df_merged.drop(columns=['first_sale_date', 'first_sale_cut', 'first_sale_price', 'first_sale_regular'])